# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Project
**_Opening a New Chinese Restaurant in Singapore_**
- Build a dataframe of neighborhoods in Singapore by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new Chinese Restaurant
***
### 1. Import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [3]:
import bs4 as bs
import urllib.request

source=urllib.request.urlopen('https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore')
soup=bs.BeautifulSoup(source,'lxml')
print(soup.title)

<title>Planning Areas of Singapore - Wikipedia</title>


In [4]:
My_table=soup.find('table',{'class':'wikitable sortable'})

In [5]:
 def parse_html_table(table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text().strip('\n'))
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text().strip('\n')
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            #for col in df:
                #try:
                    #df[col] = df[col].astype(float)
                #except ValueError:
                    #pass
            
            return df

In [6]:
import pandas as pd
df=parse_html_table(My_table)
df.head()

,Name (English),Malay,Chinese,Pinyin,Tamil,Region,Area (km2),Population[6],Density (/km2)
0,Ang Mo Kio,,宏茂桥,Hóng mào qiáo,ஆங் மோ கியோ,North-East,13.94,"165,710","12,000"
1,Bedok,*,勿洛,Wù luò,பிடோக்,East,21.69,"281,300","13,000"
2,Bishan,,碧山,Bì shān,பீஷான்,Central,7.62,"88,490","12,000"
3,Boon Lay,,文礼,Wén lǐ,பூன் லே,West,8.23,30,3.6
4,Bukit Batok,*,武吉巴督,Wǔjí bā dū,புக்கிட் பாத்தோக்,West,11.13,"144,410","13,000"


In [7]:
df.shape

(55, 9)

In [8]:
# create a list to store neighborhood data
neighborhoodList = df['Name (English)']
regionList = df['Region']

In [9]:
# create a new DataFrame from the list
sg_df = pd.DataFrame({"Neighborhood": neighborhoodList,"Region": regionList})

sg_df

,Neighborhood,Region
0,Ang Mo Kio,North-East
1,Bedok,East
2,Bishan,Central
3,Boon Lay,West
4,Bukit Batok,West
5,Bukit Merah,Central
6,Bukit Panjang,West
7,Bukit Timah,Central
8,Central Water Catchment,North
9,Changi,East


In [10]:
# print the number of rows of the dataframe
sg_df.shape

(55, 2)

### 3. Get the geographical coordinates

In [18]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Singapore'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [19]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in sg_df["Neighborhood"] ]


In [20]:
coords

[[1.3716100000000324, 103.84546000000006],
 [1.324260000000038, 103.95296000000008],
 [1.3507900000000745, 103.85110000000009],
 [1.3333300000000463, 103.70000000000005],
 [1.349520000000041, 103.75277000000006],
 [1.2841700000000742, 103.82306000000005],
 [1.3787700000000314, 103.76977000000005],
 [1.3404100000000199, 103.77221000000009],
 [1.2901800000000208, 103.85199000000006],
 [1.3551400000000626, 103.99006000000008],
 [1.3699600000000487, 103.99308000000008],
 [1.3861600000000749, 103.74618000000004],
 [1.3143800000000283, 103.76537000000008],
 [1.3771599483526997, 103.95552993392594],
 [1.3114700000000425, 103.88218000000006],
 [1.3711400000000253, 103.89144000000005],
 [1.3343700000000354, 103.74367000000007],
 [1.339490000000069, 103.70739000000003],
 [1.3333300000000463, 103.86667000000006],
 [1.4196700000000533, 103.70232000000004],
 [1.4125896854022786, 103.78968650996946],
 [1.2957900000000677, 103.89544000000006],
 [1.2785700000000588, 103.85762000000005],
 [1.3030600000

In [21]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [22]:
# merge the coordinates into the original dataframe
sg_df['Latitude'] = df_coords['Latitude']
sg_df['Longitude'] = df_coords['Longitude']

In [28]:
# check the neighborhoods and the coordinates
print(sg_df.shape)
sg_df

(55, 4)


,Neighborhood,Region,Latitude,Longitude
0,Ang Mo Kio,North-East,1.371610,103.845460
1,Bedok,East,1.324260,103.952960
2,Bishan,Central,1.350790,103.851100
3,Boon Lay,West,1.333330,103.700000
4,Bukit Batok,West,1.349520,103.752770
5,Bukit Merah,Central,1.284170,103.823060
6,Bukit Panjang,West,1.378770,103.769770
7,Bukit Timah,Central,1.340410,103.772210
8,Central Water Catchment,North,1.290180,103.851990
9,Changi,East,1.355140,103.990060


In [30]:
new_sg_df=sg_df.drop([26,52])

In [31]:
new_sg_df

,Neighborhood,Region,Latitude,Longitude
0,Ang Mo Kio,North-East,1.371610,103.845460
1,Bedok,East,1.324260,103.952960
2,Bishan,Central,1.350790,103.851100
3,Boon Lay,West,1.333330,103.700000
4,Bukit Batok,West,1.349520,103.752770
5,Bukit Merah,Central,1.284170,103.823060
6,Bukit Panjang,West,1.378770,103.769770
7,Bukit Timah,Central,1.340410,103.772210
8,Central Water Catchment,North,1.290180,103.851990
9,Changi,East,1.355140,103.990060


In [32]:
new_sg_df.shape

(53, 4)

In [33]:
# save the DataFrame as CSV file
new_sg_df.to_csv("new_sg_df.csv", index=False)

### 4. Create a map of Singapore with neighborhoods superimposed on top

In [34]:
# get the coordinates of Clementi, Singapore
address = 'Clementi, Singapore'
#address = 'Western Water Catchment, Singapore'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Clementi, Singapore {}, {}.'.format(latitude, longitude))
#print('The geograpical coordinate of Western Water Catchment, Singapore {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Clementi, Singapore 1.3140256, 103.7624098.


In [35]:
# create map of Clementi, Singapore using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(new_sg_df['Latitude'], new_sg_df['Longitude'], new_sg_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sg)  
    
map_sg

In [36]:
# save the map as HTML file
map_sg.save('map_new_sg.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [37]:
CLIENT_ID = '3HDXHJNRPN0OEI1I51O4CAALYAVD0PBSGGUC5NGZKHEFBKC5' # your Foursquare ID
CLIENT_SECRET = 'N3LZJZQJFR3GIR0IAVKVNJFPOD2BV2YV2BXFEP4IPQNJZSJQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3HDXHJNRPN0OEI1I51O4CAALYAVD0PBSGGUC5NGZKHEFBKC5
CLIENT_SECRET:N3LZJZQJFR3GIR0IAVKVNJFPOD2BV2YV2BXFEP4IPQNJZSJQ


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [38]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(new_sg_df['Latitude'], new_sg_df['Longitude'], new_sg_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [39]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4386, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Ang Mo Kio,1.37161,103.84546,A&W,1.369541,103.849043,Fast Food Restaurant
1,Ang Mo Kio,1.37161,103.84546,Bishan - Ang Mo Kio Park,1.362219,103.846250,Park
2,Ang Mo Kio,1.37161,103.84546,Old Chang Kee,1.369094,103.848389,Snack Place
3,Ang Mo Kio,1.37161,103.84546,Face Ban Mian 非板面 (Ang Mo Kio),1.372031,103.847504,Noodle House
4,Ang Mo Kio,1.37161,103.84546,Aramsa ~ The Garden Spa,1.362292,103.847602,Spa


**Let's check how many venues were returned for each neighorhood**

In [40]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Ang Mo Kio,100,100,100,100,100,100
Bedok,100,100,100,100,100,100
Bishan,100,100,100,100,100,100
Boon Lay,100,100,100,100,100,100
Bukit Batok,100,100,100,100,100,100
Bukit Merah,100,100,100,100,100,100
Bukit Panjang,76,76,76,76,76,76
Bukit Timah,100,100,100,100,100,100
Central Water Catchment,100,100,100,100,100,100


**Let's find out how many unique categories can be curated from all the returned venues**

In [41]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 297 uniques categories.


In [42]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Fast Food Restaurant', 'Park', 'Snack Place', 'Noodle House',
       'Spa', 'Supermarket', 'Asian Restaurant', 'Sandwich Place',
       'Chinese Restaurant', 'Seafood Restaurant', 'Halal Restaurant',
       'Coffee Shop', 'BBQ Joint', 'College Cafeteria', 'Bakery',
       'Burger Joint', 'Ice Cream Shop', 'Bar', 'Gym',
       'Japanese Restaurant', 'Dog Run', 'Food Court', 'Hobby Shop',
       'Dessert Shop', 'Restaurant', 'Bank', 'Modern European Restaurant',
       'Shanghai Restaurant', 'Soup Place', 'Bubble Tea Shop',
       'Pool Hall', 'Miscellaneous Shop', 'Pool', 'Thai Restaurant',
       'Indian Restaurant', 'Café', 'Vegetarian / Vegan Restaurant',
       'Cosmetics Shop', 'Office', 'Stadium', 'Australian Restaurant',
       'Market', 'Grocery Store', 'Zhejiang Restaurant', 'Golf Course',
       'Hotel', 'Beach', 'Sporting Goods Shop', 'Filipino Restaurant',
       'Electronics Store'], dtype=object)

In [43]:
# check if the results contain "Shopping Mall"
"Chinese Restaurant" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [44]:
# one hot encoding
sg_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sg_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sg_onehot.columns[-1]] + list(sg_onehot.columns[:-1])
sg_onehot = sg_onehot[fixed_columns]

print(sg_onehot.shape)
sg_onehot.head()

(4386, 298)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bay,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Event Space,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Recreation Center,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,TV Station,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Track,Track Stadium,Trail,Train Station,Tram Station,Tunnel,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietna

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [45]:
sg_grouped = sg_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(sg_grouped.shape)
sg_grouped

(53, 298)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bay,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Event Space,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Recreation Center,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,TV Station,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Track,Track Stadium,Trail,Train Station,Tram Station,Tunnel,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietna

In [46]:
len(sg_grouped[sg_grouped["Chinese Restaurant"] > 0])

36

**Create a new DataFrame for Shopping Mall data only**

In [47]:
sg_chrest = sg_grouped[["Neighborhoods","Chinese Restaurant"]]

In [48]:
sg_chrest.head()

,Neighborhoods,Chinese Restaurant
0,Ang Mo Kio,0.13
1,Bedok,0.08
2,Bishan,0.11
3,Boon Lay,0.04
4,Bukit Batok,0.07


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [49]:
# set number of clusters
kclusters = 3

sg_clustering = sg_chrest.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sg_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 2, 2, 2, 1, 2, 1, 2])

In [50]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
sg_merged = sg_chrest.copy()

# add clustering labels
sg_merged["Cluster Labels"] = kmeans.labels_

In [51]:
sg_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
sg_merged.head()

,Neighborhood,Chinese Restaurant,Cluster Labels
0,Ang Mo Kio,0.13,0
1,Bedok,0.08,0
2,Bishan,0.11,0
3,Boon Lay,0.04,2
4,Bukit Batok,0.07,2


In [52]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sg_merged = sg_merged.join(sg_df.set_index("Neighborhood"), on="Neighborhood")

print(sg_merged.shape)
sg_merged.head() # check the last columns!

(53, 6)


,Neighborhood,Chinese Restaurant,Cluster Labels,Region,Latitude,Longitude
0,Ang Mo Kio,0.13,0,North-East,1.37161,103.84546
1,Bedok,0.08,0,East,1.32426,103.95296
2,Bishan,0.11,0,Central,1.35079,103.85110
3,Boon Lay,0.04,2,West,1.33333,103.70000
4,Bukit Batok,0.07,2,West,1.34952,103.75277


In [53]:
# sort the results by Cluster Labels
print(sg_merged.shape)
sg_merged.sort_values(["Cluster Labels"], inplace=True)
sg_merged

(53, 6)


,Neighborhood,Chinese Restaurant,Cluster Labels,Region,Latitude,Longitude
0,Ang Mo Kio,0.130000,0,North-East,1.371610,103.845460
48,Toa Payoh,0.120000,0,Central,1.334480,103.851080
40,Simpang,0.094118,0,North,1.437220,103.835280
39,Serangoon,0.120000,0,North-East,1.355540,103.876600
37,Sembawang,0.113208,0,North,1.447940,103.818910
30,Paya Lebar,0.110000,0,East,1.325030,103.890490
23,Marine Parade,0.090000,0,Central,1.303060,103.907780
18,Kallang,0.150000,0,Central,1.333330,103.866670
16,Jurong East,0.090000,0,West,1.334370,103.743670
15,Hougang,0.080000,0,North-East,1.371140,103.891440


**Finally, let's visualize the resulting clusters**

In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sg_merged['Latitude'], sg_merged['Longitude'], sg_merged['Neighborhood'], sg_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [55]:
# save the map as HTML file
map_clusters.save('map_clusters_new_sg.html')

### 8. Examine Clusters

#### Cluster 0

In [56]:
sg_merged.loc[sg_merged['Cluster Labels'] == 0]

,Neighborhood,Chinese Restaurant,Cluster Labels,Region,Latitude,Longitude
0,Ang Mo Kio,0.130000,0,North-East,1.37161,103.84546
48,Toa Payoh,0.120000,0,Central,1.33448,103.85108
40,Simpang,0.094118,0,North,1.43722,103.83528
39,Serangoon,0.120000,0,North-East,1.35554,103.87660
37,Sembawang,0.113208,0,North,1.44794,103.81891
30,Paya Lebar,0.110000,0,East,1.32503,103.89049
23,Marine Parade,0.090000,0,Central,1.30306,103.90778
18,Kallang,0.150000,0,Central,1.33333,103.86667
16,Jurong East,0.090000,0,West,1.33437,103.74367
15,Hougang,0.080000,0,North-East,1.37114,103.89144


#### Cluster 1

In [57]:
sg_merged.loc[sg_merged['Cluster Labels'] == 1]

,Neighborhood,Chinese Restaurant,Cluster Labels,Region,Latitude,Longitude
41,Singapore River,0.00,1,Central,1.286670,103.852500
31,Pioneer,0.00,1,West,1.323297,103.646664
42,Southern Islands,0.00,1,Central,1.366670,103.800000
29,Pasir Ris,0.00,1,East,1.371940,103.949940
28,Outram,0.00,1,Central,1.289241,103.835002
43,Straits View,0.00,1,Central,1.279863,103.853595
44,Sungei Kadut,0.00,1,North,1.416670,103.733330
45,Tampines,0.00,1,East,1.368190,103.929480
24,Museum,0.02,1,Central,1.296720,103.849000
22,Marina South,0.00,1,Central,1.278570,103.857620


#### Cluster 2

In [58]:
sg_merged.loc[sg_merged['Cluster Labels'] == 2]

,Neighborhood,Chinese Restaurant,Cluster Labels,Region,Latitude,Longitude
38,Sengkang,0.050000,2,North-East,1.392460,103.894590
46,Tanglin,0.050000,2,Central,1.316670,103.816670
50,Western Islands,0.050000,2,West,1.330580,103.695220
3,Boon Lay,0.040000,2,West,1.333330,103.700000
12,Clementi,0.050000,2,West,1.314380,103.765370
35,Rochor,0.030000,2,Central,1.304130,103.850290
34,River Valley,0.030000,2,Central,1.293137,103.827832
33,Queenstown,0.050000,2,Central,1.299660,103.801720
11,Choa Chu Kang,0.029412,2,West,1.386160,103.746180
5,Bukit Merah,0.040000,2,Central,1.284170,103.823060


## Observations:
Most of the Chinese Restaurant are concentrated in cluster 0 and cluster 2, with the highest number in cluster 0 and moderate number in cluster 2. On the other hand, cluster 1 has very low number to totally no Chinese Restaurant in the neighborhoods. This represents a great opportunity and high potential areas to open new Chinese Restaurants as there is very little to no competition from existing Chinese Restaurants. Meanwhile, Chinese Restaurants in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of Chinese Restaurants. From another perspective, this also shows that the oversupply of Chinese Restaurants mostly happened in the central area of the city, with the suburb area still have very few Chinese Restaurants. Therefore, this project recommends property developers to capitalize on these findings to open new Chinese Restaurants in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new Chinese Restaurants in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of Chinese Restaurants and suffering from intense competition.